In [1]:
import numpy as np
import pandas as pd
from scipy.interpolate import LinearNDInterpolator
from sklearn.metrics import mean_squared_error as mse
from time import time
import os
import netCDF4 as nc
from time import time
from progressbar import progressbar
from tqdm import tqdm
tqdm.pandas()

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

In [2]:
# Functions to read CYGNSS data
def calculate_sr_value(snr, p_r, g_t, g_r, d_ts, d_sr):
    return snr - p_r - g_t - g_r - (20 * np.log10(0.19)) + (20 * np.log10(d_ts + d_sr)) + (20 * np.log10(4 * np.pi))


def compute_surface_reflectivity(df):
    df['sr'] = df.apply(
        lambda row: calculate_sr_value(row.ddm_snr, row.gps_tx_power_db_w, row.gps_ant_gain_db_i, row.sp_rx_gain,
                                       row.tx_to_sp_range, row.rx_to_sp_range), axis=1)
    return df


def calculate_hours_after_jan_value(day_of_year, ddm_timestamp):
    return (day_of_year - 1) * 24 + ddm_timestamp / (60 * 60)


def compute_hours_after_jan(df):
    df['hours_after_jan_2020'] = df.apply(
        lambda row: calculate_hours_after_jan_value(row.day_of_year, row.ddm_timestamp_utc), axis=1)
    return df


def filter_cygnss_df(df: pd.DataFrame, area: list) -> pd.DataFrame:
    """
    Filters cygnss dataframe
    :param df: pd.Dataframe
    :param area: [N, W, S, E]
    :return: pd.Dataframe
    """
    new_df = df[df['sp_lat'] < area[0]]
    new_df = new_df[new_df['sp_lat'] > area[2]]
    new_df = new_df[new_df['sp_lon'] > area[1]]
    new_df = new_df[new_df['sp_lon'] < area[3]]

    return new_df


def fuzzy_filter_cygnss_df(df: pd.DataFrame, area: list) -> pd.DataFrame:
    """
    Filters cygnss dataframe for a 10 degrees larger area than the input
    :param df: pd.Dataframe
    :param area: [N, W, S, E]
    :return: pd.Dataframe
    """
    new_df = df[df['sp_lat'] < area[0] + 10]
    new_df = new_df[new_df['sp_lat'] > area[2] - 10]
    new_df = new_df[new_df['sp_lon'] > area[1] - 10]
    new_df = new_df[new_df['sp_lon'] < area[3] + 10]

    return new_df


def get_cygnss_df(cygnss_root_path: str, days: list, area: list, fuzzy_filter=False) -> pd.DataFrame:
    """
    If you want 1st to 3rd of August, days should be [1, 2, 3]
    :param cygnss_root_path: path to the root folder with cygnss data
    :param days: days list, e.g. [1, 2, 3]
    :param area: [N, W, S, E]
    :param fuzzy_filter: boolean. Whether or not to use fuzzy filter.
    :return: pd.Dataframe
    """

    file_start = 'raw_main_df_2020_08_'
    file_ending = 'of31.csv'

    cygnss_df = pd.DataFrame()

    for day in days:
        current_path = cygnss_root_path + file_start + str(day) + file_ending
        if fuzzy_filter:
            current_cygnss_df = compute_surface_reflectivity(fuzzy_filter_cygnss_df(pd.read_csv(current_path), area))
        else:
            current_cygnss_df = compute_surface_reflectivity(filter_cygnss_df(pd.read_csv(current_path), area))
        current_cygnss_df = compute_hours_after_jan(current_cygnss_df)
        cygnss_df = cygnss_df.append(current_cygnss_df)

    return cygnss_df


In [3]:
# Functions to read SMAP data
def get_smap(path: str, printing=False):
    ds = nc.Dataset(path)
    sm = ds['Soil_Moisture_Retrieval_Data_AM']

    latitudes = []
    longitudes = []
    moistures = []
    times = []
    qfs = []
    landcover_01 = []
    landcover_02 = []
    landcover_03 = []
    roughness = []
    surface_temp = []
    vo = []
    veg_wat_cont = []

    for lat in progressbar(range(len(sm['latitude']))):
        for long in range(len(sm['longitude'][lat])):
            latitudes.append(sm['latitude'][lat][long])
            longitudes.append(sm['longitude'][lat][long])
            moistures.append(sm['soil_moisture'][lat][long])
            times.append(sm['tb_time_utc'][lat][long])
            qfs.append(sm['retrieval_qual_flag'][lat][long])
            landcover_01.append(sm['landcover_class.Bands_01'][lat][long])
            landcover_02.append(sm['landcover_class.Bands_02'][lat][long])
            landcover_03.append(sm['landcover_class.Bands_03'][lat][long])
            roughness.append(sm['roughness_coefficient'][lat][long])
            surface_temp.append(sm['surface_temperature'][lat][long])
            vo.append(sm['vegetation_opacity'][lat][long])
            veg_wat_cont.append(sm['vegetation_water_content'][lat][long])

    # df = pd.DataFrame.from_dict({'lat': latitudes, 'long': longitudes, 'time': times, 'smap_sm': moistures})
    df = pd.DataFrame.from_dict({'lat': latitudes, 'long': longitudes, 'time': times, 'smap_sm': moistures,
                                 'retrieval_qfs': qfs, 'surface_roughness': roughness,
                                 'surface_temp': surface_temp, 'vegetation_opacity': vo,
                                 'vegetation_water_content': veg_wat_cont, 'landcover_class_01': landcover_01,
                                 'landcover_class_02': landcover_02, 'landcover_class_03': landcover_03})

    # Filter out missing values
    smap_df = df[df['smap_sm'] != -9999.0]

    if len(smap_df) > 0 and printing:
        print('Number of missing values:', len(df) - len(smap_df))
        print('Number of data points with value:', len(smap_df))
        index = list(smap_df['smap_sm']).index(max(list(smap_df['smap_sm'])))
        print("Peak SM value:", list(smap_df['smap_sm'])[index])
        print("Peak SM value at: (" + str(list(smap_df['lat'])[index]) + ", " + str(list(smap_df['long'])[index]) + ")")

    return smap_df


def conv(t):
    try:
        return pd.Timestamp(t)
    except:
        return pd.Timestamp(t.split('.')[0] + '.000Z')


def convert_time(df: pd.DataFrame) -> pd.DataFrame:
    ref_date = pd.Timestamp('2020-01-01T00:00:00.000Z')

    df['time'] = df['time'].apply(lambda t: conv(t))
    df['time'] = df['time'].apply(lambda t: (t - ref_date).days * 24 + (t - ref_date).seconds / 3600)
    return df


def get_smap_df(root_dir: str, year: int, convert_time_hours=True) -> pd.DataFrame:
    first = True
    all_paths = []
    for subdir, dirs, files in os.walk(root_dir):
        for file in files:
            if not first:
                all_paths.append(os.path.join(subdir, file))
            else:
                first = False

    smap_df = pd.DataFrame()

    for path in all_paths:
        path_split = path.split('_')
        current_year = int(path_split[4][:4])

        if current_year == year:
            current_df = get_smap(path)
            smap_df = smap_df.append(current_df)

    if convert_time_hours:
        smap_df = convert_time(smap_df)

    return smap_df


In [4]:
# Helper functions for the main program
def create_interval_list(interval: int, min_value: int, max_value: int) -> list:
    interval_list = []

    while min_value <= max_value:
        interval_list.append(min_value)
        min_value += interval

    return interval_list

def interpolate(df: pd.DataFrame, target_value, lat_name='lat', long_name='long',
                time_name='time') -> LinearNDInterpolator:
    coordinates = list(zip(list(df[time_name]), list(df[lat_name]), list(df[long_name])))
    target = df[target_value]
    interpolation_function = LinearNDInterpolator(coordinates, target)
    return interpolation_function

def filter_nan_smap(df, target='sm'):
    try:
        df[target] = df[target].apply(lambda x: x.item(0))
    except:
        print('SMAP_SM value was already of type: float')
    df = df.dropna()
    return df

In [5]:
# Variables
target_val = 'sp_inc_angle'
cygnss_root_path = "/Volumes/DACOTA HDD/Semester Project CSV/CYGNSS 2020-08/"
smap_root_path = "/Users/vegardhaneberg/Desktop/Masters Thesis/Code/Master/Data/SMAP/India first two weeks of August"
days = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
#      [  N     W      S      E]
area = [27.2, 80.32, 21.81, 88.29]
interval = 20
min_value = 0
max_value = 60
fuzzy = False
step_size = 2
incidence_classes = list(range(min_value, max_value, step_size))
statistics_dict = {}  

In [6]:
# Read CYGNSS
start = time()
cygnss_df = get_cygnss_df(cygnss_root_path, days, area, fuzzy_filter=fuzzy)
print('Done reading ' + str(len(cygnss_df)) + ' CYGNSS items in ' + str(round(time() - start, 0)) + ' sec')

FileNotFoundError: [Errno 2] No such file or directory: '/Volumes/DACOTA HDD/Semester Project CSV/CYGNSS 2020-08/raw_main_df_2020_08_1of31.csv'

In [ ]:
# Read SMAP
start = time()
smap_df = get_smap_df(smap_root_path, 2020, convert_time_hours=True)
print('Done reading ' + str(len(smap_df)) + ' SMAP items in ' + str(round(time() - start, 0)) + ' sec')

In [ ]:
def print_data_statistics(smap_df, cygnss_df):
    smap_lat_min = smap_df['lat'].min()
    smap_lat_max = smap_df['lat'].max()
    smap_long_min = smap_df['long'].min()
    smap_long_max = smap_df['long'].max()
    smap_time_min = smap_df['time'].min()
    smap_time_max = smap_df['time'].max()

    print('SMAP items:', len(smap_df))
    print('SMAP lat min:', smap_lat_min)
    print('SMAP lat max:', smap_lat_max)
    print('SMAP long min:', smap_long_min)
    print('SMAP long max:', smap_long_max)
    print('SMAP time min:', smap_time_min)
    print('SMAP time max:', smap_time_max)
    print('\n')
    print("CYGNSS items:", len(cygnss_df))
    print('CYGNSS lat min:', cygnss_df['sp_lat'].min())
    print('CYGNSS lat max:', cygnss_df['sp_lat'].max())
    print('CYGNSS long min:', cygnss_df['sp_lon'].min())
    print('CYGNSS long max:', cygnss_df['sp_lon'].max())
    print('CYGNSS time min:', cygnss_df['hours_after_jan_2020'].min())
    print('CYGNSS time max:', cygnss_df['hours_after_jan_2020'].max())
    

In [ ]:
# cygnss_df = cygnss_df[cygnss_df['sp_lat'] > smap_lat_min]
# cygnss_df = cygnss_df[cygnss_df['sp_lat'] < smap_lat_max]
# cygnss_df = cygnss_df[cygnss_df['sp_lon'] > smap_long_min]
# cygnss_df = cygnss_df[cygnss_df['sp_lon'] < smap_long_max]
# cygnss_df = cygnss_df[cygnss_df['hours_after_jan_2020'] > smap_time_min]
# cygnss_df = cygnss_df[cygnss_df['hours_after_jan_2020'] < smap_time_max]


In [ ]:
# Interpolate soil moisture, vegetation opacity and surface roughness
inter_function_inc = interpolate(smap_df, 'smap_sm', 'lat', 'long', 'time')
cygnss_df['sm'] = cygnss_df.apply(lambda row: inter_function_inc(row.hours_after_jan_2020, row.sp_lat, row.sp_lon), axis=1)
cygnss_df = filter_nan_smap(cygnss_df)

inter_function_vo = interpolate(smap_df, 'vegetation_opacity', 'lat', 'long', 'time')
cygnss_df['vegetation_opacity'] = cygnss_df.apply(lambda row: inter_function_vo(row.hours_after_jan_2020, row.sp_lat, row.sp_lon), axis=1)
cygnss_df = filter_nan_smap(cygnss_df, target='vegetation_opacity')

inter_function_surface_roughness = interpolate(smap_df, 'surface_roughness', 'lat', 'long', 'time')
cygnss_df['surface_roughness'] = cygnss_df.apply(lambda row: inter_function_surface_roughness(row.hours_after_jan_2020, row.sp_lat, row.sp_lon), axis=1)
cygnss_df = filter_nan_smap(cygnss_df, target='surface_roughness')


In [ ]:
# Incidence angle
for group in incidence_classes:

    # Filter the target value
    current_cygnss = cygnss_df[cygnss_df[target_val] >= group]
    if not group == incidence_classes[len(incidence_classes) - 1]:
        current_cygnss = current_cygnss[current_cygnss[target_val] < group + interval]

    print('Processing values: ' + str(group) + ' to ' + str(group + interval) + '    Length of cygnss: ' + 
          str(len(current_cygnss)) + '    Current group: ' + str(group))
    
    corr = current_cygnss['sm'].corr(current_cygnss['sr'])
    statistics_dict[group] = corr


In [ ]:
# Plot incidence angle
fig = plt.figure()
ax = plt.axes()

ax.plot(list(statistics_dict.keys()), list(statistics_dict.values()))

In [ ]:
# Vegetation Opacity
target_val = 'vegetation_opacity'
#      [  N     W      S      E  ]
area = [27.2, 80.32, 21.81, 88.29]
interval = 0.1
min_value = 20
max_value = 70
classes = None
fuzzy = False
step_size = 5

classes_100 = list(range(min_value, max_value, step_size))
classes_vegetation_opacity = []
for c in classes_100:
    classes_vegetation_opacity.append(c/100)

statistics_dict_vo = {}

In [ ]:
for group in classes_vegetation_opacity:
    current_cygnss = cygnss_df[cygnss_df[target_val] >= group]
    if not group == classes_vegetation_opacity[len(classes_vegetation_opacity) - 1]:
        current_cygnss = current_cygnss[current_cygnss[target_val] < group + interval]

    print('Processing values: ' + str(group) + ' to ' + str(round(group + interval, 1)) + '    Length of cygnss: ' + 
          str(len(current_cygnss)))
    
    corr = current_cygnss['sm'].corr(current_cygnss['sr'])
    statistics_dict_vo[group] = corr

In [ ]:
fig = plt.figure()
ax = plt.axes()

ax.plot(list(statistics_dict_vo.keys()), list(statistics_dict_vo.values()))

In [ ]:
# Surface Roughness
target_val = 'surface_roughness'
#      [  N     W      S      E  ]
area = [27.2, 80.32, 21.81, 88.29]
min_value = smap_df['surface_roughness'].min()
max_value = smap_df['surface_roughness'].max()
bins = 20
classes_surface_roughness = []
fuzzy = False
step_size = (max_value - min_value)/bins

for i in range(bins):
    classes_surface_roughness.append(min_value + i*step_size)

statistics_dict_surface_roughness = {}

In [ ]:
for group in classes_surface_roughness:
    current_cygnss = cygnss_df[cygnss_df[target_val] >= group]
    if not group == classes_surface_roughness[len(classes_surface_roughness) - 1]:
        current_cygnss = current_cygnss[current_cygnss[target_val] < group + interval]

    print('Processing values: ' + str(group) + ' to ' + str(round(group + interval, 1)) + '    Length of cygnss: ' + 
          str(len(current_cygnss)))
    
    corr = current_cygnss['sm'].corr(current_cygnss['sr'])
    statistics_dict_surface_roughness[group] = corr

In [ ]:
fig = plt.figure()
ax = plt.axes()

ax.plot(list(statistics_dict_surface_roughness.keys()), list(statistics_dict_surface_roughness.values()))